<a href="https://colab.research.google.com/github/Edward-TL/amazon_scraper/blob/master/Products_Control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Start

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


#Import

In [3]:
#For the system
import os

#Manage of time
from datetime import datetime, timedelta
from pytz import timezone
import time

#Manage of files
import pandas as pd
import csv
import numpy as np

#scrap
from bs4 import BeautifulSoup
from openpyxl.workbook import Workbook
import requests

#For SQL
!pip install psycopg2
import psycopg2

!pip install sqlalchemy
import sqlalchemy as sql

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


#Go to folder

Hay que considerar que se trabajara con las bases de datos (cualquier formato) y con las carpetas de control de product. Por ello, es que se inicia en la carpeta que permite seleccionar esto. Recordando:
*  Masters: Se encuentran las bases de datos, organizadas por tipo de archivo.
*  Origin: Primeros experimentos, se encuentran los archivos originales por consulta, divididos por país y categoría del país.
*   Products: Dónde se pretende almacenar las base de características de productos.

In [4]:
%cd 'drive/My Drive/Colab Notebooks/4SS/4SS_db/testing'
!ls

/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing
Masters  Origin  Products


#Tools

##Centered Len

#Count if

In [5]:
def count_if(array, value):
    count = 0
    for element in array:
        if element == value:
            count += 1
    return count

##Tools

In [6]:
def mode(array):
    elements = [None]*len(array)
    counter = [None]*len(array)

    for element in array:

        #Check the element
        searcher = 0
        while elements[searcher] != element and elements[searcher] != None and searcher < (len(array) - 1):
            searcher += 1
        
        if elements[searcher] == element:
            counter[searcher] += 1
        
        elif elements[searcher] == None:
            elements[searcher] = element
            counter[searcher] = 1

    max = 0
    c = 0
    while counter[c] != None and c<(len(counter)-1):
        if counter[c] >= max:
            max = counter[c]
            max_position = c
        c += 1

    mode = elements[max_position]
    return mode

#Standarize dates

In [7]:
def standarize_dates(date):
    check = False
    try:
        date = datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S.%f')
        check = True
    except ValueError:
        try:
            date = datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S')
            check = True
        except:
            try:
                date = datetime.strptime(str(date), '%Y-%m-%d %H_%M')
                check = True
            except:
                pass
    if check == True:
        date = date.strftime('%Y-%m-%d %H:%M')
    
    return date

#Lab

##Extract Products

###Name Products

In [8]:
entry = 'Masters/mx/parquet/mx-master_db_music.parquet'

#Get de Nanmes
df = pd.read_parquet(entry)
df['time'] = df['time'].apply(standarize_dates)
df = df.sort_values(by="time")
product_names_raw = df['Product Names'].tolist()
#Remember, this gets de None Value
products = list(set(product_names_raw))
None_cases = count_if(product_names_raw, None)
products.remove(None)

In [20]:
products

['Muzz (Vinyl)',
 'Matenme Porque Me Muero',
 'I Carus',
 'Royal Blood [Importado]',
 'Tributo a Sabina/ Ni Tan Joven Ni Tan Viejo',
 'Midnight Marauders (Vinyl)[Importado]',
 'Malibu Nights (Vinyl)',
 'After Hours',
 'Chromatica - Empaque de pasta dura- Incluye: CD + álbum + bonus tracks',
 'A Star Is Born (Original Soundtrack)',
 'Tainted Lunch (Vinyl)',
 'Unplugged in Ny (Vinyl) [Importado]',
 'To Bring You My Love - Demos (Vinyl)',
 'Three Sixty',
 'Kind of Magic (Vinyl)[Importado]',
 '(Whats the Story) Morning Glory [Importado]',
 'Trick Or Treat [Importado]',
 'Thriller (Vinyl)',
 'Slippery When Wet (Vinyl)',
 'Chops [Importado]',
 'Birds of Fire [Importado]',
 'MTV Unplugged: El Desconecte',
 'CIGALA CANTA A MEXICO',
 'Stadium Arcadium',
 'The Queen is Dead (2017 Master) & Additional Recordings (Audio CD)',
 'Random Access Memories (Vinyl) [Importado]',
 'Sobre El Amor y Sus Efectos Secundarios (Audio CD)',
 'Weezer (Blue Album) (Vinyl)',
 'Goats Head Soup',
 'X 100PRE (Vinyl)',

##Scrap Dates

In [9]:
dates_raw = df['time'].tolist()
dates_time = set(dates_raw)

just_dates = []
for dt in dates_time:
    try:
        date = datetime.strptime(dt,'%Y-%m-%d %H:%M').date()
    except:
        date = datetime.strptime(dt,'%Y-%m-%d %H_%M').date()

    if not date in just_dates:
        just_dates.append(date)

just_dates.sort()

#Obtaining All Mode Ranks in table

In [10]:
products_dict = {products[n]:n for n in range(len(products))}
# products_dict

In [15]:
ranks = df['Rank'].tolist()
day_arrays = [None] * (len(just_dates) - 1)
zero_rank_day_array = [None] * len(products)
rank_day_array = [None] * len(products)

d_counter = 0
first_date = just_dates[0]

for r in range(len(ranks)):
    
    #Day that we are
    try:
        date_check = datetime.strptime(dates_raw[r],'%Y-%m-%d %H:%M').date()
    except:
        date_check = datetime.strptime(dates_raw[r],'%Y-%m-%d %H_%M').date()

    if date_check != first_date:
        day_arrays[d_counter] = rank_day_array
        rank_day_array = [None] * len(products)
        if d_counter < (len(just_dates)-1):
            d_counter += 1
            first_date = just_dates[d_counter]

    if date_check == first_date:
        # if date_check == just_dates[0]:
        #     print(date_check)
        rank = ranks[r]
        #What product are we talking about
        product_name = product_names_raw[r]
        if product_name != None:
            product_position = products_dict[product_name]
            if rank_day_array[product_position] == None:
                array = []
                rank_day_array[product_position] = array
                array_rank = rank_day_array[product_position]
                array_rank.append(rank)
            else:
                array_rank = rank_day_array[product_position]
                array_rank.append(rank)
        
            # print(f'{product_name} Position: {product_position} rank: {rank} -> {rank_day_array[product_position]}')

for ranked_day in day_arrays:
    print(ranked_day)

print('\n')

for ranked_day in day_arrays:
    for r in range(len(ranked_day)):
        product = ranked_day[r]
        if product != None:
            if len(product) > 1:
                moda = mode(ranked_day[r])
                ranked_day[r] = moda
            # elif len(product) == 0:
            #     ranked_day[r] = None
            elif len(product) == 1:
                ranked_day[r] = product[0]

for ranked_day in day_arrays:
    print(ranked_day)


[None, None, None, None, [11.0, 11.0, 11.0, 11.0, 11.0, 11.0], [4.0, 4.0, 4.0, 4.0, 10.0, 10.0, 10.0, 10.0, 10.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0], None, [44.0, 31.0, 31.0, 31.0, 31.0, 31.0, 31.0], [3.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0], None, None, None, None, None, None, None, None, None, [47.0, 47.0, 47.0, 47.0, 47.0, 47.0], None, None, None, None, None, [26.0, 26.0, 26.0, 26.0, 26.0, 26.0], [37.0], None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, [40.0, 40.0, 40.0, 40.0, 40.0, 40.0], None, None, None, None, None, [29.0], None, [41.0], None, None, None, None, None, None, [12.0, 28.0, 28.0, 28.0, 28.0, 13.0, 13.0, 13.0, 13.0, 13.0, 27.0, 27.0, 27.0, 27.0, 27.0, 27.0], None, None, None, [49.0, 49.0, 49.0, 49.0, 49.0], None, None, [38.0, 38.0, 38.0, 38.0, 38.0, 38.0], None, [20.0, 8.0, 20.0, 8.0, 8.0, 20.0, 20.0, 8.0, 15.0, 12.0, 15.0, 12.0, 12.0, 15.0, 15.0, 12.0, 15.0, 12.0, 5.0, 6

##Pandas Queries

###About a Product

In [43]:
product_query = products[335]

In [38]:
Select_query = 'Product Names'

In [44]:
df[df[Select_query] == product_query]

,time,Rank,Product Names,Image urls,Product links,Stars,Reviews,Authors/Company,Edition/Console,Price_std_or_min,Max_prices
40,2020-08-13 10:53,41.0,Guardians of the Galaxy (Vinyl) [Importado],https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Guardians-Galaxy-Vin...,4.7,1723.0,Tyler Bates,Vinil,737.0,None
40,2020-08-13 11:43,41.0,Guardians of the Galaxy (Vinyl) [Importado],https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Guardians-Galaxy-Vin...,4.7,1723.0,Tyler Bates,Vinil,737.0,None
40,2020-08-13 12:29,41.0,Guardians of the Galaxy (Vinyl) [Importado],https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Guardians-Galaxy-Vin...,4.7,1723.0,Tyler Bates,Vinil,737.0,None


###Best Seller of the day

In [46]:
rank_1_df = df[df['Rank'] == 1]
rank_1_df

,time,Rank,Product Names,Image urls,Product links,Stars,Reviews,Authors/Company,Edition/Console,Price_std_or_min,Max_prices
3750,2020-07-31 00:38,1.0,FINE LINE,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/FINE-LINE-HARRY-STYL...,4.8,775.0,HARRY STYLES,Audio CD,170.0,None
2950,2020-07-31 11:00,1.0,Chromatica - Empaque de pasta dura- Incluye: C...,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Chromatica-Empaque-p...,4.6,190.0,Lady Gaga,Audio CD,429.8,None
2750,2020-07-31 11:28,1.0,Chromatica - Empaque de pasta dura- Incluye: C...,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Chromatica-Empaque-p...,4.6,190.0,Lady Gaga,Audio CD,429.8,None
2900,2020-07-31 11:49,1.0,Chromatica - Empaque de pasta dura- Incluye: C...,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Chromatica-Empaque-p...,4.6,190.0,Lady Gaga,Audio CD,429.8,None
2500,2020-07-31 12:10,1.0,Chromatica - Empaque de pasta dura- Incluye: C...,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Chromatica-Empaque-p...,4.6,191.0,Lady Gaga,Audio CD,429.8,None
...,...,...,...,...,...,...,...,...,...,...,...
0,2020-08-13 18:25,1.0,Folklore,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Folklore-Taylor-Swif...,5.0,9.0,Taylor Swift,Audio CD,338.7,None
0,2020-08-13 19:12,1.0,Folklore,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Folklore-Taylor-Swif...,5.0,9.0,Taylor Swift,Audio CD,338.7,None
0,2020-08-13 19:59,1.0,Folklore,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Folklore-Taylor-Swif...,5.0,9.0,Taylor Swift,Audio CD,338.7,None
0,2020-08-13 20:46,1.0,Folklore,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.mx/Folklore-Taylor-Swif...,5.0,9.0,Taylor Swift,Audio CD,338.7,None


In [49]:
MX__COL_tz = 'America/Mexico_City'
timezone_MXCOL = timezone(MX__COL_tz) 
today_datetime = datetime.now(timezone_MXCOL)
today = today_datetime.date()
today

datetime.date(2020, 8, 14)

In [54]:
yesterday = today - timedelta(days=1)
rank_1_df[rank_1_df['time'] == yesterday]

,time,Rank,Product Names,Image urls,Product links,Stars,Reviews,Authors/Company,Edition/Console,Price_std_or_min,Max_prices
